Figure 1

In [ ]:
#A simple schematic: defender dot, ball landing dot, two arrows (defender orientation and ball direction), and the angle between them (orientation error).

def plot_figure1_concept():
    fig, ax = plt.subplots(figsize=(6, 6))

    # Simple coordinate system
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)

    # Defender and ball positions (toy example)
    x_def, y_def = 3, 3
    x_ball, y_ball = 8, 7

    # Plot defender and ball
    ax.scatter([x_def], [y_def], s=80)
    ax.text(x_def, y_def - 0.4, "Defender", ha="center")

    ax.scatter([x_ball], [y_ball], s=80)
    ax.text(x_ball, y_ball + 0.4, "Ball landing\npoint", ha="center")

    # Defender orientation (toy direction)
    defender_dir_deg = 20.0
    defender_dir_rad = np.deg2rad(defender_dir_deg)
    ax.arrow(
        x_def, y_def,
        2.5 * np.cos(defender_dir_rad),
        2.5 * np.sin(defender_dir_rad),
        head_width=0.2, length_includes_head=True
    )
    ax.text(x_def + 2.6 * np.cos(defender_dir_rad),
            y_def + 2.6 * np.sin(defender_dir_rad),
            "Facing\ndirection", ha="left", va="bottom")

    # Ball direction vector
    dx = x_ball - x_def
    dy = y_ball - y_def
    ball_angle_rad = np.arctan2(dy, dx)
    ax.arrow(
        x_def, y_def,
        2.5 * np.cos(ball_angle_rad),
        2.5 * np.sin(ball_angle_rad),
        head_width=0.2, length_includes_head=True
    )
    ax.text(x_def + 2.6 * np.cos(ball_angle_rad),
            y_def + 2.6 * np.sin(ball_angle_rad),
            "Ball\nvector", ha="left", va="bottom")

    # Orientation error (just annotate the angle)
    defender_dir_deg = (defender_dir_deg + 360) % 360
    ball_dir_deg = (np.rad2deg(ball_angle_rad) + 360) % 360
    error = abs((defender_dir_deg - ball_dir_deg + 180) % 360 - 180)

    ax.text(1.0, 9.2,
            f"Orientation error ≈ {error:.1f}°",
            fontsize=11)

    ax.set_xlabel("Field X (yards)")
    ax.set_ylabel("Field Y (yards)")
    ax.set_title("Figure 1 – Patience Score Concept Diagram")

    ax.set_aspect("equal", adjustable="box")
    plt.tight_layout()
    plt.show()


Figure 2

In [ ]:
#Orientation error vs time for one defender in one play, with a vertical line at the reaction frame.

Assumes you can subset sorted_df to a single play+defender and pass it in.

def compute_orientation_error(df_play):
    """
    Given a DataFrame for ONE defender in ONE play with columns:
    x, y, dir, ball_land_x, ball_land_y
    return a 1D array of orientation error in degrees [0, 180].
    """
    x = df_play["x"].astype(float).to_numpy()
    y = df_play["y"].astype(float).to_numpy()
    d = df_play["dir"].astype(float).to_numpy()
    bx = float(df_play["ball_land_x"].iloc[0])
    by = float(df_play["ball_land_y"].iloc[0])

    dx = bx - x
    dy = by - y
    ball_angle = np.rad2deg(np.arctan2(dy, dx))
    ball_angle = (ball_angle + 360.0) % 360.0

    diff = (d - ball_angle + 180.0) % 360.0 - 180.0
    error = np.abs(diff)
    return error


def find_reaction_frame(errors_deg, threshold_deg=3.0):
    """
    errors_deg: np.array of orientation error in degrees.
    Returns index of first frame where error improves by >= threshold
    relative to frame 0, or None if none found.
    """
    e0 = errors_deg[0]
    improvement = e0 - errors_deg
    idx_candidates = np.where(improvement >= threshold_deg)[0]
    return int(idx_candidates[0]) if len(idx_candidates) > 0 else None


def plot_figure2_reaction_timeline(df_play, seconds_per_frame=0.1):
    """
    df_play: ONE defender in ONE play, sorted by frame_id.
    Must have columns: frame_id, x, y, dir, ball_land_x, ball_land_y.
    """
    df_play = df_play.sort_values("frame_id")
    errors = compute_orientation_error(df_play)
    reaction_idx = find_reaction_frame(errors, threshold_deg=3.0)

    t = np.arange(len(errors)) * seconds_per_frame

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(t, errors)
    ax.set_xlabel("Time since ball release (s)")
    ax.set_ylabel("Orientation error (degrees)")
    ax.set_title("Figure 2 – Reaction timeline for a single play")

    if reaction_idx is not None:
        ax.axvline(
            x=reaction_idx * seconds_per_frame,
            linestyle="--"
        )
        ax.text(
            reaction_idx * seconds_per_frame,
            errors[reaction_idx],
            "  Reaction",
            va="bottom"
        )

    plt.tight_layout()
    plt.show()

# Example: pick the first play in your sorted_df
example_keys = sorted_df[["game_id", "play_id", "coverage_defender_id"]].dropna().iloc[0]
g = sorted_df[
    (sorted_df["game_id"] == example_keys["game_id"]) &
    (sorted_df["play_id"] == example_keys["play_id"]) &
    (sorted_df["coverage_defender_id"] == example_keys["coverage_defender_id"])
]

plot_figure2_reaction_timeline(g)


Figure 3

In [ ]:
#Distance between defender and ball landing point over time for one play, with a horizontal catch-radius line.

def compute_distance_to_ball(df_play):
    x = df_play["x"].astype(float).to_numpy()
    y = df_play["y"].astype(float).to_numpy()
    bx = float(df_play["ball_land_x"].iloc[0])
    by = float(df_play["ball_land_y"].iloc[0])
    dists = np.sqrt((bx - x)**2 + (by - y)**2)
    return dists


def plot_figure3_distance_curve(df_play, catch_radius_yd, seconds_per_frame=0.1):
    """
    df_play: ONE defender in ONE play, sorted by frame_id.
    catch_radius_yd: scalar catch radius in yards (from your model).
    """
    df_play = df_play.sort_values("frame_id")
    dists = compute_distance_to_ball(df_play)
    t = np.arange(len(dists)) * seconds_per_frame

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(t, dists, label="Distance to ball")
    ax.axhline(catch_radius_yd, linestyle="--")
    ax.set_xlabel("Time since ball release (s)")
    ax.set_ylabel("Distance to ball (yards)")
    ax.set_title("Figure 3 – Distance-to-ball curve for a single play")

    plt.tight_layout()
    plt.show()


Figure 4

In [ ]:
#n/a

Figure 5

In [ ]:
#Distribution of scores

def plot_figure5_patience_hist(player_scores):
    valid = player_scores["patience_score"].dropna()

    fig, ax = plt.subplots(figsize=(7, 4))
    ax.hist(valid, bins=30)
    ax.set_xlabel("Patience Score (z-score, more negative = more patient)")
    ax.set_ylabel("Number of defenders")
    ax.set_title("Figure 5 – Final Patience Score distribution")

    plt.tight_layout()
    plt.show()
plot_figure5_patience_hist(player_scores)


Figure 6

In [ ]:
#Hypothesis test

Figure 7

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# -------------------------------------------------------------------
# EXAMPLE DATA
# -------------------------------------------------------------------
# Replace this block with a merge between your player_scores DataFrame
# and a table of PFF-ranked corners.
data = [
    ("Jaylon Johnson",      1, -0.95),
    ("DaRon Bland",         2,  0.10),
    ("Sauce Gardner",       3, -1.10),
    ("Trent McDuffie",      4, -0.80),
    ("Charvarius Ward",     5, -0.55),
    ("Devon Witherspoon",   6, -0.75),
    ("Kendall Fuller",      8, -0.65),
    ("Christian Benford",   9, -0.20),
    ("Derek Stingley Jr.", 10, -0.45),
    ("Isaac Yiadom",       11, -0.60),
    ("Rasul Douglas",      12,  0.05),
    ("Michael Carter II",  13,  1.44),
]

df = pd.DataFrame(data, columns=["player", "pff_rank", "patience_score"])

# Sort so the most patient (most negative) are at the bottom/top consistently
df_sorted = df.sort_values("patience_score")

# -------------------------------------------------------------------
# PLOT
# -------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(8, 6))

ax.barh(df_sorted["player"], df_sorted["patience_score"])

# Vertical line at 0 to show league-average / standardized center
ax.axvline(0, linestyle="--")

ax.set_xlabel("Patience Score (more negative = more patient)")
ax.set_title("Figure 7 – Patience Scores for PFF Top Cornerbacks")

plt.tight_layout()
plt.show()


In [ ]:
##Optional add on

pff_corners = [
    "Jaylon Johnson", "DaRon Bland", "Sauce Gardner", "Trent McDuffie",
    "Charvarius Ward", "Devon Witherspoon", "Kendall Fuller",
    "Christian Benford", "Derek Stingley Jr.", "Isaac Yiadom",
    "Rasul Douglas", "Michael Carter II"
]

df = (
    player_scores
    .loc[player_scores["player_name"].isin(pff_corners),
         ["player_name", "patience_score"]]
    .rename(columns={"player_name": "player"})
)

# Optionally add PFF rank if you want to sort by that instead:
pff_rank_map = {
    "Jaylon Johnson": 1,
    "DaRon Bland": 2,
    "Sauce Gardner": 3,
    "Trent McDuffie": 4,
    "Charvarius Ward": 5,
    "Devon Witherspoon": 6,
    "Kendall Fuller": 8,
    "Christian Benford": 9,
    "Derek Stingley Jr.": 10,
    "Isaac Yiadom": 11,
    "Rasul Douglas": 12,
    "Michael Carter II": 13,
}
df["pff_rank"] = df["player"].map(pff_rank_map)

# Then you can plot exactly as above using df instead of the example df.


Figure 8

In [ ]:
def plot_figure8_man_vs_zone_top20(player_scores):
    """
    Plots a scatter of man vs zone patience scores but ONLY for the
    top 20 most patient defenders (most negative final Patience Score).

    player_scores must contain:
      - patience_score
      - patience_score_man
      - patience_score_zone
      - player_name
    """

    # Select top 20 most patient (most negative values)
    df = (
        player_scores
        .dropna(subset=["patience_score_man", "patience_score_zone", "patience_score"])
        .sort_values("patience_score")  # ascending = more negative first
        .head(20)
    )

    fig, ax = plt.subplots(figsize=(7, 6))

    ax.scatter(df["patience_score_man"], df["patience_score_zone"])

    # Label each point with player name
    for _, row in df.iterrows():
        ax.text(
            row["patience_score_man"],
            row["patience_score_zone"],
            row["player_name"],
            fontsize=8,
            ha="left",
            va="bottom"
        )

    # Reference lines
    ax.axvline(0, linestyle="--")
    ax.axhline(0, linestyle="--")

    ax.set_xlabel("Man patience z-score (more negative = more patient)")
    ax.set_ylabel("Zone patience z-score (more negative = more patient)")
    ax.set_title("Figure 8 – Top 20 Most Patient Defenders: Man vs Zone Profiles")

    plt.tight_layout()
    plt.show()

plot_figure8_man_vs_zone_top20(player_scores)
